In [107]:
!ls

libc++abi.dylib: terminating with uncaught exception of type std::runtime_error: Couldn't close file


In [75]:
from fastai.vision import *
from torch.optim import Optimizer
from Optimizers.lookahead import *
from torchcontrib.optim import SWA
import wandb
from wandb.fastai import WandbCallback

In [65]:
PROJECT_PATH = Path.cwd()
MODELS = PROJECT_PATH/'models'

MODELS.mkdir(exist_ok=True)

In [105]:
def print_all(*args):
    for i in args:
        print(i)

In [66]:
def get_mnist_data(bs=16):
    PATH = Path("/Users/akash/.fastai/data/mnist_png")
    data = (ImageList.from_folder(path=PATH)
            .split_by_folder(train='training', valid='testing')
            .label_from_folder()
            .databunch(bs=bs))
    return data

In [67]:
def get_cifar10_data(bs=16):
    pad4 = RandTransform(tfm=pad, kwargs={'padding':4, 'mode':'zeros'}, is_random=False, p=1.0, use_on_y=False)
    crop32 = RandTransform(tfm=crop, kwargs={'row_pct':(0.4,0.6), 'col_pct':(0.4,0.6), 'size':32}, p=1.0, use_on_y=False)
    tfms = [pad4, crop32]
    PATH = Path("/Users/akash/.fastai/data/cifar10")
    data = (ImageList.from_folder(path=PATH)
            .split_by_folder(train='train', valid='test')
            .label_from_folder()
            .transform((tfms, []))
            .databunch(bs=bs))
    return data

In [68]:
data = get_mnist_data()
data.show_batch()

def get_simple_cnn(pretrained=False):
    return simple_cnn([3, 4, 2])

learn = cnn_learner(data=data, base_arch=get_simple_cnn, opt_func=lookahead.LookaheadSGD)
learn.metrics.append(accuracy)
learn.fit(1, 0.003)
learn.save(MODELS/'mnist_sample.pkl')
learn.load(MODELS/'mnist_sample.pkl')

In [54]:
wandbRecorder = partial(WandbCallback, input_type='images')

In [110]:
optimizers = [LookaheadSGD, optim.SGD, optim.AdamW, optim.RMSprop, LookaheadAdamW] # Along with SWA which needs to be separately dealt with
experiment_names = {LookaheadSGD:'Lookahead_SGD', optim.SGD:'SGD', optim.AdamW:'AdamW', 
                        optim.RMSprop:'RMSProp', LookaheadAdamW:'LookaheadAdamW'}

lookahead_sgd_params = dict(alpha=0.5, k=5) # lr = 0.1
sgd_params = dict(momentum=0.9, weight_decay=0.001) # lr = 0.05
adamW_params = dict(weight_decay=0.3) # lr = 0.001
rmsprop_params = dict(weight_decay=0.001) # lr = 0.01
lookahead_adamw_params = dict(weight_decay=0.3, k=5, alpha=0.5) # lr = 0.1

params_list = [lookahead_sgd_params, sgd_params, adamW_params, rmsprop_params, lookahead_adamw_params]

params_dict = {LookaheadSGD:lookahead_sgd_params, optim.SGD:sgd_params, optim.AdamW:adamW_params,
              optim.RMSprop:rmsprop_params, LookaheadAdamW:lookahead_adamw_params}
lr_dict = {LookaheadSGD:0.1, optim.SGD:0.05, optim.AdamW:0.001,
              optim.RMSprop:0.01, LookaheadAdamW:0.1}
epochs_dict = {LookaheadSGD:20, optim.SGD:20, optim.AdamW:20,
              optim.RMSprop:20, LookaheadAdamW:20}

In [94]:
def fit_and_record(epochs=None, lr=None, opt_func=lookahead.LookaheadSGD, experiment_name=None, 
                   data=None, base_arch=models.resnet18, pretrained=False):
    np.random.seed(42)
    # Initialize Training
    if experiment_name is None:
        experiment_name = experiment_names[opt_func]
    if data is None:
        data = get_cifar10_data(128)
    if epochs is None:
        epochs = epochs_dict[opt_func]
    if lr is None:
        lr = lr_dict[opt_func]
    # Setting optimizer parameters correctly
    opt_func = partial(opt_func, **params_dict[opt_func])
    
    print_all(data, opt_func, epochs, lr, experiment_name, base_arch)
    
    wandb.init(project='lookahead', name=experiment_name)
    learn = cnn_learner(data=data, base_arch=base_arch, opt_func=opt_func, pretrained=pretrained, callback_fns=Wand)
    learn.metrics.append(accuracy)
    learn.fit(epochs, lr)
    learn.save(MODELS/experiment_name)

In [ ]:
for optimizer in optimizers:
    fit_and_record(opt_func=optimizer)